In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import looptools.loopmath as lm
from looptools.mokulaserlock import MokuLaserLock
from looptools.component import Component

In [ ]:
sps = 80e6 # System clock rate
frfr = np.logspace(np.log10(1e-3), np.log10(1e6), int(1e5)) # Fourier frequency array (Hz)
frfr = frfr[0:-1]

# Definition of the plant Component:
nume = [-27391.4746505128605349, 28991.6861562978592701, 27391.5753081338189077, -28991.5850488191608747]
deno = [1.0, -2.9907469440381682, 2.9815121426943869, -0.9907651980332260, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Plant = Component("Plant", sps, nume=nume, deno=deno)

# Moku laser lock parameters:
ll_parameters = {
'Plant': Plant, # looptools Component specifying the plant
'Amp_reference': 1.0, # Mixer local oscillator amplitude (Vpp)
'Amp_input': 1.0, # Beatnote amplitude (Vpp)
'LPF_cutoff': 0.25e6, # Butterworth LPF cutoff frequency 
'LPF_n': 4, # Butterworth LPF, number of cascaded stages
'Cshift': 14, # Gain reduction stage, number of bits for LeftBitShift
'Kp_db': -5, # P-gain (dB)
'f_I': 800, # First integrator crossover frequency (Hz)
'f_II': None, # Second integrator crossover frequency (Hz)
'n_reg': 100, # DSP delay component (number of registers)
'off': [None],
'f_trans': None
}

ll = MokuLaserLock(
    Plant = ll_parameters['Plant'],
    Amp_reference = ll_parameters['Amp_reference'],
    Amp_input = ll_parameters['Amp_input'],
    LPF_cutoff = ll_parameters['LPF_cutoff'],
    LPF_n = ll_parameters['LPF_n'],
    Cshift = ll_parameters['Cshift'],
    Kp_db = ll_parameters['Kp_db'],
    f_I = ll_parameters['f_I'],
    f_II = ll_parameters['f_II'],
    n_reg = ll_parameters['n_reg'],
    off = ll_parameters['off']
)

ugf, margin = lm.get_margin(ll.Gf(f=frfr), frfr, deg=True) # compute UGF and phase margin
print(f"Unity gain frequency = {ugf:.4e} Hz; Phase margin = {margin:.4f} degrees")

In [ ]:
ll.block_diagram(filename='block-diagram.tex')

In [ ]:
ax = ll.components_dict['Mixer'].bode_plot(frfr, dB=False)
plt.show()

In [ ]:
ax = ll.components_dict['LPF'].bode_plot(frfr, dB=True)
plt.show()

In [ ]:
ax = ll.components_dict['Servo'].bode_plot(frfr, dB=True)
plt.show()

In [ ]:
ax = ll.components_dict['Plant'].bode_plot(frfr, dB=True)
plt.show()

In [ ]:
ax = ll.bode_plot(frfr, dB=True)
ax[0].axvline(x=ugf, ls='--', c='gray', lw=1)
ax[1].axvline(x=ugf, ls='--', c='gray', lw=1)
plt.show()

In [ ]:
ax = ll.nyquist_plot(np.logspace(3,6,int(1e5)), which='G', logy=True, logx=True, arrow_frequency=ugf, critical_point=True, label=False)
plt.show()